# Demo

This notebook contains a simple demo on how to work with the code. Before running this code you should also make sure to download the data.

In [1]:
!curl -L 'https://www.dropbox.com/s/6bkbw6v269dyfie/data.tar' -o data.tar && tar -xvf data.tar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   116    0   116    0     0    106      0 --:--:--  0:00:01 --:--:--   106
100   343  100   343    0     0    208      0  0:00:01  0:00:01 --:--:--   208
100   534    0   534    0     0    177      0 --:--:--  0:00:03 --:--:--   496
100 66.9M  100 66.9M    0     0  3446k      0  0:00:19  0:00:19 --:--:-- 4798k
mesh/
mesh/heart01.msh
mesh/heart02.msh


Just make sure that the meshes are downloaded by listing them

In [2]:
!ls mesh

heart01.msh  heart02.msh


Now we need to make sure that we can import the fuctions from the `code` folder. We can do this by adding the `code` folder to `sys.path`. 

In [3]:
import sys
sys.path.insert(0, "../code")

In the dataset, there are two meshes. Let us pick the smallest heart which is heart number 1

In [4]:
heart_nr = 1

Let us specify the path to gmsh file and make sure ths file exist

In [5]:
from pathlib import Path

msh_file = Path("mesh") / f"heart{heart_nr:02}.msh"

assert msh_file.is_file()

## Pre-processing

First step is to convert the gmsh file to dolfin format. We can do this with the `convert_mesh` function from the `pre-processing` module

In [6]:
import pre_processing

outfile =  Path(f"heart{heart_nr:02}.h5")
pre_processing.convert_mesh(msh_file=msh_file, outfile=outfile)

Converting 'msh' file to dolfin xdmf format
Saved to heart01.h5


## Fiber generation 

We can now take the mesh in dolfin format and generate the fiber orientations using the ldrb algorithm.

In [7]:
import run_fiber_generation


microstructure_path = Path(f"microstructure{heart_nr:02}.h5")
run_fiber_generation.generate_fibers(outfile=outfile, microstructure_path=microstructure_path)

Calculating scalar fields
Compute scalar laplacian solutions with the markers: 
base: 1
epi: 2
lv: 3
rv: 4
  Num vertices: 1571
  Num cells: 4717
Solving linear variational problem.
  Apex coord: (345.54, 1232.91, 170.73)
Solving linear variational problem.
Solving Laplace equation
 epi = 1, lv, rv = 0
Solving linear variational problem.
 lv = 1, epi, rv = 0
Solving linear variational problem.
 rv = 1, epi, lv = 0
Solving linear variational problem.
Solving linear variational problem.

Calculating gradients
Compute fiber-sheet system
Angles: 
alpha: 
 endo_lv: 60
 epi_lv: -60
 endo_septum: 60
 epi_septum: -60
 endo_rv: 60
 epi_rv: -60
beta: 
 endo_lv: 0
 epi_lv: 0
 endo_septum: 0
 epi_septum: 0
 endo_rv: 0
 epi_rv: 0
Microstructure saved to microstructure01.h5


## Post processing

Finally we will run the postprocessing script where we convert the fiber fields to a file we can visualize in paraview, and compare some feautres against the features preseted in an artificial paper


In [10]:
from postprocess import generate_fiber_xdmf_file

fiber_path = Path(f"fiber_{heart_nr:02}.xdmf")
features = generate_fiber_xdmf_file(
    outfile=outfile, 
    microstructure_path=microstructure_path, 
    fiber_path=fiber_path
)
print(features)

Saved fibers to fiber_01.xdmf
{'size': 4713, 'min': -0.9986740049294516, 'max': 0.9997579018328593, 'mean': 0.08134810142530069, 'std': 0.5715906050031196}
